# Scope of this workbook

Here, we want to actually model the whole thing for different datasets. 

In [22]:
# Imports
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

import datetime

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


#Settings
pd.set_option("display.max_rows", 70)
pd.set_option("display.max_columns", 101)

# Processed Dataset

In [23]:
full_df = pd.read_pickle('../data/finalized/full_df.pkl')

#full_df.describe()

In [24]:
# temp 

# additional drop of variables that are not available at the start of day 

full_df.drop([
    'trans_time',
    'ground_delay',
    'act_groundtime_before',
    'act_groundtime_new'
], axis = 1, inplace = True)


In [25]:
full_df.head()

,leg_no,ac_registration_x,Ac Type Code,sched_trans_time,Sched Groundtime,flt_event_number,leg,block_delay,day_of_week,hour_of_day_dep,hour_of_day_arr,minute_of_day_dep,Crewchange,sched_groundtime_before,Crewchange_before,sched_groundtime_new,rows_to_drop_grounddelay
9,272024970,ECLBAX,320,0,95.0,1,New Jessica-East Carmen,-9.0,5,3,6,205,both,95.0,first flt of day,95.0,0
120,272022230,ECLBAX,320,60,75.0,2,East Carmen-South Nathaniel,-9.0,5,8,10,500,no change,95.0,both,75.0,0
198,272212848,ECLBAX,320,75,80.0,3,South Nathaniel-East Carmen,-14.0,5,11,14,710,both,75.0,no change,80.0,0
283,271997824,ECLBAX,320,70,50.0,4,East Carmen-Joneshaven,-11.0,5,15,17,935,no change,80.0,both,50.0,0
439,272337586,ECLBAX,320,45,45.0,1,East Carmen-Joneshaven,2.0,6,6,8,410,no change,45.0,first flt of day,45.0,0


## Encoding & Train/Test split

In [26]:
# Creating dummy variables for all categorical variables
# Note: Onehotencoder is the better solution, however for simplicity let's use pandas for the moment

# Get object columns
full_df_objectcolumns = full_df.select_dtypes(include = ['object', 'category'])
varlist = full_df_objectcolumns.columns.values.tolist()

# get dummies
full_df_encoded = pd.get_dummies(full_df, columns = varlist, drop_first = True)

#filter = ['ac']
#print(str([i for i in full_df_encoded.columns if 'Route' not in i ])+"\n")

full_df_encoded.shape

(7465, 318)

In [27]:
# We split the forecast in two different forecasts - one for ground delay and one for block delay

# dropping the ground delay filter row
full_df_encoded_blockdelay = full_df_encoded.drop(['rows_to_drop_grounddelay'], axis = 1)
X_train_blockdelay_encoded, X_test_blockdelay_encoded, y_train_blockdelay_encoded, y_test_blockdelay_encoded = train_test_split(
    full_df_encoded_blockdelay.drop(['block_delay'], axis = 1), full_df_encoded_blockdelay['block_delay'], test_size=0.33, random_state=42)

# Filtering out rows which are skewing ground delay prediction
#full_df_encoded_grounddelay = full_df_encoded[full_df_encoded['rows_to_drop_grounddelay'] == 0]

#X_train_grounddelay_encoded, X_test_grounddelay_encoded, y_train_grounddelay_encoded, y_test_grounddelay_encoded = train_test_split(
#    full_df_encoded_grounddelay.drop(['ground_delay'], axis = 1), full_df_encoded_grounddelay['ground_delay'], test_size=0.33, random_state=42)

'''#Hinnerk****************************************************************************************************

# dropping the ground delay filter row
full_df_encoded_blockdelay = full_df_encoded#.drop(['rows_to_drop_grounddelay'], axis = 1)
X_train_blockdelay_encoded, X_test_blockdelay_encoded, y_train_blockdelay_encoded, y_test_blockdelay_encoded = train_test_split(
    full_df_encoded_blockdelay.drop(['block_delay'], axis = 1), full_df_encoded_blockdelay['block_delay'], test_size=0.33, random_state=42)

# Filtering out rows which are skewing ground delay prediction
full_df_encoded_grounddelay = full_df_encoded#[full_df_encoded['rows_to_drop_grounddelay']<1]

X_train_grounddelay_encoded, X_test_grounddelay_encoded, y_train_grounddelay_encoded, y_test_grounddelay_encoded = train_test_split(
    full_df_encoded_grounddelay.drop(['ground_delay'], axis = 1), full_df_encoded_grounddelay['ground_delay'], test_size=0.33, random_state=42)
'''

"#Hinnerk****************************************************************************************************\n\n# dropping the ground delay filter row\nfull_df_encoded_blockdelay = full_df_encoded#.drop(['rows_to_drop_grounddelay'], axis = 1)\nX_train_blockdelay_encoded, X_test_blockdelay_encoded, y_train_blockdelay_encoded, y_test_blockdelay_encoded = train_test_split(\n    full_df_encoded_blockdelay.drop(['block_delay'], axis = 1), full_df_encoded_blockdelay['block_delay'], test_size=0.33, random_state=42)\n\n# Filtering out rows which are skewing ground delay prediction\nfull_df_encoded_grounddelay = full_df_encoded#[full_df_encoded['rows_to_drop_grounddelay']<1]\n\nX_train_grounddelay_encoded, X_test_grounddelay_encoded, y_train_grounddelay_encoded, y_test_grounddelay_encoded = train_test_split(\n    full_df_encoded_grounddelay.drop(['ground_delay'], axis = 1), full_df_encoded_grounddelay['ground_delay'], test_size=0.33, random_state=42)\n"

## Linear Regression


In [28]:
# Blocktime
lr_block = LinearRegression()

lr_block.fit(X_train_blockdelay_encoded, y_train_blockdelay_encoded)
lr_block.predict(X_test_blockdelay_encoded)

# Groundtime
#lr_ground = LinearRegression()

#lr_ground.fit(X_train_grounddelay_encoded, y_train_grounddelay_encoded)
#lr_ground.predict(X_test_grounddelay_encoded)

array([ -7.63557883, -11.15157442,  -6.12492434, ...,  -7.19986977,
        -7.16011714,   1.40093464])

In [29]:
# Blocktime
r2_blockdelay = lr_block.score(X_test_blockdelay_encoded, y_test_blockdelay_encoded)
rmse_blockdelay = np.sqrt(mean_squared_error(y_test_blockdelay_encoded, lr_block.predict(X_test_blockdelay_encoded)))
print("The r^2 for Block delay is " + str(round(r2_blockdelay,4)))
print("The RMSE for Block delay is " + str(round(rmse_blockdelay,2)) + " minutes.")
print("\n")

# Groundtime
#r2_grounddelay = lr_ground.score(X_test_grounddelay_encoded, y_test_grounddelay_encoded)
#rmse_grounddelay = np.sqrt(mean_squared_error(y_test_grounddelay_encoded, lr_ground.predict(X_test_grounddelay_encoded)))
#print("The r^2 for Ground delay is " + str(round(r2_grounddelay,4)))
#print("The RMSE for Ground delay is " + str(round(rmse_grounddelay,2)) + " minutes.")

The r^2 for Block delay is 0.2364
The RMSE for Block delay is 6.08 minutes.




## Setup of Modelling

In [30]:
# Imports 
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


In [31]:
%%time
# Initialize estimators

#reg1 = LinearRegression()
reg2 = Ridge()
#reg3 = Lasso()
reg4 = GradientBoostingRegressor()
#reg5 = SVR()

# Initialize hyperparameters for each dictionary
#param1 = {}

param2 = {}
param2['regressor__alpha'] = [0,1,2,3]#[x for x in np.linspace(0,2,num = 4)]
param2['regressor'] = [reg2]

param4 = {}
param4['regressor__n_estimators'] = [40,50,60] #[10,50,100,150,250]#[10,50,100,150,200,250]
param4['regressor__max_depth'] = [10,12,14]#,18,20]#,18]#,22] #[2,3,6,8,10,12,18] #[2,3,4,6,8,10,12,15,18]
param4['regressor__max_leaf_nodes'] = [28,30,32]#[6,8,12,15,19,25]#,18] #[6,8,12,None] #[6,8,12,15,19,25,None]
param4['regressor'] = [reg4]


# Create Pipeline
pipeline = Pipeline([('regressor', reg2)])
params = [param2, param4]

# Train grid search model
gs_block = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error', verbose=1).fit(X_train_blockdelay_encoded,y_train_blockdelay_encoded)
#gs_ground = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error', verbose=1).fit(X_train_grounddelay_encoded,y_train_grounddelay_encoded)
#gs_block = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, verbose=1).fit(X_train_blockdelay_encoded,y_train_blockdelay_encoded)
#gs_ground = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, verbose=1).fit(X_train_grounddelay_encoded,y_train_grounddelay_encoded)
print(gs_block.best_estimator_, gs_block.best_params_)
#print(gs_ground.best_estimator_, gs_ground.best_params_)

#Evaluation
# Blocktime
r2_blockdelay = gs_block.best_estimator_.score(X_test_blockdelay_encoded, y_test_blockdelay_encoded)
rmse_blockdelay = np.sqrt(mean_squared_error(y_test_blockdelay_encoded, gs_block.best_estimator_.predict(X_test_blockdelay_encoded)))
print("The r^2 for Block delay is " + str(round(r2_blockdelay,4)))
print("The RMSE for Block delay is " + str(round(rmse_blockdelay,2)) + " minutes.")
print("\n")

# Groundtime
#r2_grounddelay = gs_ground.best_estimator_.score(X_test_grounddelay_encoded, y_test_grounddelay_encoded)
#rmse_grounddelay = np.sqrt(mean_squared_error(y_test_grounddelay_encoded, gs_ground.best_estimator_.predict(X_test_grounddelay_encoded)))
#print("The r^2 for Ground delay is " + str(round(r2_grounddelay,4)))
#print("The RMSE for Ground delay is " + str(round(rmse_grounddelay,2)) + " minutes.")



Fitting 3 folds for each of 31 candidates, totalling 93 fits
Pipeline(steps=[('regressor',
                 GradientBoostingRegressor(max_depth=14, max_leaf_nodes=30,
                                           n_estimators=50))]) {'regressor': GradientBoostingRegressor(max_depth=14, max_leaf_nodes=30, n_estimators=50), 'regressor__max_depth': 14, 'regressor__max_leaf_nodes': 30, 'regressor__n_estimators': 50}
The r^2 for Block delay is 0.2526
The RMSE for Block delay is 6.01 minutes.


CPU times: total: 5.89 s
Wall time: 1min 40s


# Evaluation of Best Model

In [21]:
feature_importances = gs_block.best_estimator_.steps[0][1].feature_importances_

d = {'Features': X_train_blockdelay_encoded.columns, 'Importance': feature_importances}
eval = pd.DataFrame(data = d).sort_values(by = 'Importance', ascending = False)
eval.head(50)

,Features,Importance
79,Ac Type Code_DH4,0.207968
0,leg_no,0.111547
7,minute_of_day_dep,0.045998
80,Ac Type Code_E95,0.042834
237,leg_Lake Lawrencechester-Lake Richardberg,0.030766
4,day_of_week,0.026784
331,route_East Carmen-East Holly,0.024839
8,sched_groundtime_before,0.019323
6,hour_of_day_arr,0.019123
1,sched_trans_time,0.018805


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
# Create dataframe to save model evaluation parameters
eval = pd.DataFrame(columns= ['Group', 'Model', 'Parameters', 'R^2 test', 'RMSE test', 'R^2 train', 'RMSE train'])

# Create ridge regression models with best alpha values
rid_block = Ridge(alpha=2, fit_intercept=True)
rid_ground = Ridge(alpha=2, fit_intercept=True)

# Fit models to training data
rid_block.fit(X_train_blockdelay_encoded, y_train_blockdelay_encoded)
rid_ground.fit(X_train_grounddelay_encoded, y_train_grounddelay_encoded)

# Predict values for train and test data
rid_pred_block_train = rid_block.predict(X_train_blockdelay_encoded)
rid_pred_ground_train = rid_ground.predict(X_train_grounddelay_encoded)

rid_pred_block_test = rid_block.predict(X_test_blockdelay_encoded)
rid_pred_ground_test = rid_ground.predict(X_test_grounddelay_encoded)

# Save r^2 and RMSE for both models in dataframe for later comparison
eval = eval.append({
    'Group': 'Block',
    'Model': 'Ridge',
    'R^2 test': r2_score(y_test_blockdelay_encoded, rid_pred_block_test),
    'RMSE test': mean_squared_error(y_test_blockdelay_encoded, rid_pred_block_test, squared=False),
    'R^2 train': r2_score(y_train_blockdelay_encoded, rid_pred_block_train),
    'RMSE train': mean_squared_error(y_train_blockdelay_encoded, rid_pred_block_train, squared=False)
    }, ignore_index=True)
 
eval = eval.append({
    'Group': 'Ground',
    'Model': 'Ridge',
    'R^2 test': r2_score(y_test_grounddelay_encoded, rid_pred_ground_test),
    'RMSE test': mean_squared_error(y_test_grounddelay_encoded, rid_pred_ground_test, squared=False),
    'R^2 train': r2_score(y_train_grounddelay_encoded, rid_pred_ground_train),
    'RMSE train': mean_squared_error(y_train_grounddelay_encoded, rid_pred_ground_train, squared=False),
    }, ignore_index=True)

eval.round(decimals=3)

C:\Users\PC Paul\AppData\Local\Temp\ipykernel_18312\1589020009.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval = eval.append({
C:\Users\PC Paul\AppData\Local\Temp\ipykernel_18312\1589020009.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval = eval.append({


,Group,Model,Parameters,R^2 test,RMSE test,R^2 train,RMSE train
0,Block,Ridge,NaN,0.320295,5.734718,0.3692,5.378322
1,Ground,Ridge,NaN,0.672487,10.071233,0.718203,9.374951


## Gradient Boosted Tree

In [ ]:
# Gradientboosting as base classifier

# Blocktime
base_model_blockdelay = GradientBoostingRegressor()

base_model_blockdelay.fit(X_train_blockdelay_encoded, y_train_blockdelay_encoded)
base_model_blockdelay.predict(X_test_blockdelay_encoded)


# Groundtime

base_model_grounddelay = GradientBoostingRegressor()

base_model_grounddelay.fit(X_train_grounddelay_encoded, y_train_grounddelay_encoded)
base_model_grounddelay.predict(X_test_grounddelay_encoded)


array([44.41952744, 13.83831236, 25.63892846, ..., 29.71946156,
       -0.62027353,  9.36199806])

In [ ]:
# Blocktime
r2_blockdelay = base_model_blockdelay.score(X_test_blockdelay_encoded, y_test_blockdelay_encoded)
rmse_blockdelay = np.sqrt(mean_squared_error(y_test_blockdelay_encoded, base_model_blockdelay.predict(X_test_blockdelay_encoded)))
print("The r^2 for Block delay is " + str(round(r2_blockdelay,4)))
print("The RMSE for Block delay is " + str(round(rmse_blockdelay,2)) + " minutes.")
print("\n")

# Groundtime
r2_grounddelay = base_model_grounddelay.score(X_test_grounddelay_encoded, y_test_grounddelay_encoded)
rmse_grounddelay = np.sqrt(mean_squared_error(y_test_grounddelay_encoded, base_model_grounddelay.predict(X_test_grounddelay_encoded)))
print("The r^2 for Ground delay is " + str(round(r2_grounddelay,4)))
print("The RMSE for Ground delay is " + str(round(rmse_grounddelay,2)) + " minutes.")

The r^2 for Block delay is 0.8795
The RMSE for Block delay is 6.55 minutes.


The r^2 for Ground delay is 0.902
The RMSE for Ground delay is 5.53 minutes.
